In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import polars as pl
import json
from polars import selectors as cs
from utils import *


import rcs_cataloging as rcc # --> probably need to build first

### TODO: Implement version of DerivedTime (assignTime.m) here

In [3]:
json_file = '/Users/claysmyth/Desktop/Starr/RCS_Hypno_Data/RCS02/RCS02R/8_18_21/Session1629269941495/DeviceNPC700404H/RawDataTD.json'

In [4]:
with open(json_file, 'r') as file:
    data = json.load(file)

In [5]:
td_data = data[0]['TimeDomainData']

In [6]:
df = rcc.loop_and_table_td_data(td_data)

In [7]:
df = df.with_columns(
    [
        pl.col('localTime').str.to_datetime("%Y-%m-%d %H:%M:%S%.3f"),
        # Chunk interval is how much time each chunk of data represents (overshot by 1 sample because of indexing, corrected by "closed='left'" call below)
        # (pl.col('sample_rate').replace({0: 4, 1: 2, 2: 1}) * pl.col('channel_0').list.len()).alias('chunk_interval'),
        # pl.col('sample_rate').replace({0: '4ms', 1: '2ms', 2: '1ms'}).alias('sample_interval'),
        pl.col('sample_rate').replace({0: 250, 1: 500, 2: 1000}).alias('samplerate'),
        pl.col('channel_0').list.len().alias('packetsizes'),
    ]
)#.with_columns(
#     # Will overshoot by 1 sample, corrected by "closed='left'" call below
#     localEndTime = pl.col('localTime') + pl.duration(milliseconds=pl.col('chunk_interval'))
# )

In [8]:
df = df.explode(pl.col('^channel.*$'))

In [9]:
df = df.with_columns(
    pl.when(
        (pl.col('localTime').diff() != 0) | (pl.col('localTime').diff().is_null()),
    ).then(
        pl.col(['timestamp', 'PacketGenTime', 'systemTick', 'PacketRxUnixTime', 'samplerate', 'packetsizes'])
    ).otherwise(pl.lit(None))
)

In [15]:
df_2 = assign_time(df.to_pandas())

Identifying and removing bad packets
Chunking data
Determining start time of each chunk
Number of chunks: 1567
Number of chunks with time calculated from PacketGenTime: 1567
Shifting chunks to align with sampling rate


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [7]:
df = pl.concat([df_tmp.with_columns(
                    (pl.datetime_ranges(pl.col('localTime'), pl.col('localEndTime'), interval=interval, closed='left')
                    ).alias('localTime_vec')
                )
                for interval, df_tmp in df.partition_by('sample_interval',as_dict=True).items()], 
    how='vertical').sort('localTime')
# Select only the columns we want...
                

/var/folders/1d/gj64tt_x46x6d62lws668qnr0000gn/T/ipykernel_12681/1998063042.py:5: DeprecationWarning: `partition_by(..., as_dict=True)` will change to always return tuples as dictionary keys. Pass `by` as a list to silence this warning, e.g. `partition_by(['sample_interval'], as_dict=True)`.
  for interval, df_tmp in df.partition_by('sample_interval',as_dict=True).items()],


In [8]:
df.head()

localTime,timestamp,PacketGenTime,PacketRxUnixTime,sample_rate,channel_0,channel_1,channel_2,channel_3,chunk_interval,sample_interval,RateInHz,localEndTime,localTime_vec
datetime[ms],i64,i64,i64,i64,list[f64],list[f64],list[f64],list[f64],i64,str,i64,datetime[ms],list[datetime[ms]]
2021-08-18 00:00:50.624,677376004,-62135568000000,1629270050624,0,"[-0.135137, -0.099997, … -0.165613]","[-0.151189, -0.115506, … -0.177001]","[-0.056013, -0.042119, … -0.055138]","[-0.020173, -0.005965, … -0.019305]",48,"""4ms""",250,2021-08-18 00:00:50.672,"[2021-08-18 00:00:50.624, 2021-08-18 00:00:50.628, … 2021-08-18 00:00:50.668]"
2021-08-18 00:00:50.701,677376004,-62135568000000,1629270050701,0,"[-0.130365, -0.123858, … -0.121905]","[-0.143705, -0.138065, … -0.138825]","[-0.057325, -0.067609, … -0.076799]","[-0.018221, -0.029175, … -0.03796]",52,"""4ms""",250,2021-08-18 00:00:50.753,"[2021-08-18 00:00:50.701, 2021-08-18 00:00:50.705, … 2021-08-18 00:00:50.749]"
2021-08-18 00:00:50.748,677376004,-62135568000000,1629270050748,0,"[-0.163444, -0.167349, … -0.166915]","[-0.180147, -0.183183, … -0.184268]","[-0.071438, -0.073407, … -0.070891]","[-0.035357, -0.034381, … -0.03373]",52,"""4ms""",250,2021-08-18 00:00:50.800,"[2021-08-18 00:00:50.748, 2021-08-18 00:00:50.752, … 2021-08-18 00:00:50.796]"
2021-08-18 00:00:50.806,677376004,-62135568000000,1629270050806,0,"[-0.216588, -0.145549, … -0.110951]","[-0.233182, -0.161926, … -0.126677]","[-0.074173, -0.079971, … -0.075595]","[-0.037418, -0.041647, … -0.037092]",48,"""4ms""",250,2021-08-18 00:00:50.854,"[2021-08-18 00:00:50.806, 2021-08-18 00:00:50.810, … 2021-08-18 00:00:50.850]"
2021-08-18 00:00:50.860,677376004,-62135568000000,1629270050860,0,"[-0.143163, -0.123966, … -0.139367]","[-0.160516, -0.138608, … -0.153466]","[-0.068594, -0.057872, … -0.058091]","[-0.029826, -0.018871, … -0.020932]",48,"""4ms""",250,2021-08-18 00:00:50.908,"[2021-08-18 00:00:50.860, 2021-08-18 00:00:50.864, … 2021-08-18 00:00:50.904]"


In [9]:
list_chans = cs.by_name(['channel_0', 'channel_1', 'channel_2', 'channel_3', 'localTime_vec'])
df = df.explode(list_chans)

In [10]:
df.head()

localTime,timestamp,PacketGenTime,PacketRxUnixTime,sample_rate,channel_0,channel_1,channel_2,channel_3,chunk_interval,sample_interval,RateInHz,localEndTime,localTime_vec
datetime[ms],i64,i64,i64,i64,f64,f64,f64,f64,i64,str,i64,datetime[ms],datetime[ms]
2021-08-18 00:00:50.624,677376004,-62135568000000,1629270050624,0,-0.135137,-0.151189,-0.056013,-0.020173,48,"""4ms""",250,2021-08-18 00:00:50.672,2021-08-18 00:00:50.624
2021-08-18 00:00:50.624,677376004,-62135568000000,1629270050624,0,-0.099997,-0.115506,-0.042119,-0.005965,48,"""4ms""",250,2021-08-18 00:00:50.672,2021-08-18 00:00:50.628
2021-08-18 00:00:50.624,677376004,-62135568000000,1629270050624,0,-0.103468,-0.120278,-0.044088,-0.009761,48,"""4ms""",250,2021-08-18 00:00:50.672,2021-08-18 00:00:50.632
2021-08-18 00:00:50.624,677376004,-62135568000000,1629270050624,0,-0.083403,-0.098804,-0.063889,-0.026897,48,"""4ms""",250,2021-08-18 00:00:50.672,2021-08-18 00:00:50.636
2021-08-18 00:00:50.624,677376004,-62135568000000,1629270050624,0,-0.084379,-0.099021,-0.077893,-0.038719,48,"""4ms""",250,2021-08-18 00:00:50.672,2021-08-18 00:00:50.640


In [11]:
df.sort('localTime_vec')

localTime,timestamp,PacketGenTime,PacketRxUnixTime,sample_rate,channel_0,channel_1,channel_2,channel_3,chunk_interval,sample_interval,RateInHz,localEndTime,localTime_vec
datetime[ms],i64,i64,i64,i64,f64,f64,f64,f64,i64,str,i64,datetime[ms],datetime[ms]
2021-08-18 00:00:50.624,677376004,-62135568000000,1629270050624,0,-0.135137,-0.151189,-0.056013,-0.020173,48,"""4ms""",250,2021-08-18 00:00:50.672,2021-08-18 00:00:50.624
2021-08-18 00:00:50.624,677376004,-62135568000000,1629270050624,0,-0.099997,-0.115506,-0.042119,-0.005965,48,"""4ms""",250,2021-08-18 00:00:50.672,2021-08-18 00:00:50.628
2021-08-18 00:00:50.624,677376004,-62135568000000,1629270050624,0,-0.103468,-0.120278,-0.044088,-0.009761,48,"""4ms""",250,2021-08-18 00:00:50.672,2021-08-18 00:00:50.632
2021-08-18 00:00:50.624,677376004,-62135568000000,1629270050624,0,-0.083403,-0.098804,-0.063889,-0.026897,48,"""4ms""",250,2021-08-18 00:00:50.672,2021-08-18 00:00:50.636
2021-08-18 00:00:50.624,677376004,-62135568000000,1629270050624,0,-0.084379,-0.099021,-0.077893,-0.038719,48,"""4ms""",250,2021-08-18 00:00:50.672,2021-08-18 00:00:50.640
…,…,…,…,…,…,…,…,…,…,…,…,…,…
2021-08-18 06:07:32.925,677398006,1629292052855,1629292052925,0,-0.087633,-0.100214,-0.12034,-0.08503,52,"""4ms""",250,2021-08-18 06:07:32.977,2021-08-18 06:07:32.957
2021-08-18 06:07:32.925,677398006,1629292052855,1629292052925,0,-0.110517,-0.1244,-0.12034,-0.086006,52,"""4ms""",250,2021-08-18 06:07:32.977,2021-08-18 06:07:32.961
2021-08-18 06:07:32.925,677398006,1629292052855,1629292052925,0,-0.085681,-0.101407,-0.125482,-0.091104,52,"""4ms""",250,2021-08-18 06:07:32.977,2021-08-18 06:07:32.965


In [19]:
tmp = df.sort('localTime_vec').with_columns(
    pl.col('localTime_vec').diff().alias('diff')
).get_column('diff')

In [21]:
tmp.value_counts()

diff,count
duration[ms],u32
4ms,3338482
162ms,1
45ms,1367
165ms,3
78ms,212
…,…
60ms,676
171ms,1
39ms,1908


In [12]:
df.sort('localTime_vec').group_by_dynamic('localTime_vec', every='1s', period='1s', start_by='datapoint').agg(
    [
        pl.col('localTime').first(),
        pl.col('timestamp').first(),
        pl.col('PacketGenTime').first(),
        pl.col('PacketRxUnixTime').first(),
        pl.col('RateInHz').first(),
        pl.col('channel_0'),
        pl.col('channel_1'),
        pl.col('channel_2'),
        pl.col('channel_3'),
        pl.col('localTime_vec').count().alias('epoch_length'),
    ]
).get_column('epoch_length').unique_counts()

epoch_length
u32
655
575
1795
1350
1692
…
1
1
1


## Settings

In [31]:
device_settings_file = '/Users/claysmyth/Desktop/Starr/RCS_Hypno_Data/RCS02/RCS02R/8_18_21/Session1629269941495/DeviceNPC700404H/DeviceSettings.json'
with open(device_settings_file, 'r') as file:
    settings_data = json.load(file)

In [117]:
settings = process_device_settings(settings_data)

{'elecValue': 63, 'electrodeType': 1, 'isOff': False, 'reserved1': 0}
{'RecordInfo': {'ApiVer': '1.6.0.0', 'DeviceId': 'NPC700404H', 'HostUnixTime': 1629269956365, 'SessionId': '1629269941495'}, 'GeneralData': {'batteryStatus': 91, 'daysUntilEos': 2446, 'deviceSerialNumber': [78, 80, 67, 55, 48, 48, 52, 48, 52, 72], 'deviceTime': {'seconds': 677375910}, 'hybridSerialNumber': [0, 2, 153, 24, 193, 154], 'implantDate': {'seconds': 605318400}, 'isEos': False, 'isEri': False, 'isInCPSession': False, 'isMriModeOn': False, 'isOvervoltage': False, 'isTherapyUnavailable': False, 'telMDeviceId': [64, 0, 84, 200, 198, 12], 'therapyStatusData': {'activeGroup': 3, 'therapyStatus': 1}, 'type': 1}}
{'RecordInfo': {'ApiVer': '1.6.0.0', 'DeviceId': 'NPC700404H', 'HostUnixTime': 1629269956651, 'SessionId': '1629269941495'}, 'GeneralData': {'batteryStatus': 91, 'daysUntilEos': 2446, 'deviceSerialNumber': [78, 80, 67, 55, 48, 48, 52, 48, 52, 72], 'deviceTime': {'seconds': 677375910}, 'hybridSerialNumber':

/var/folders/1d/gj64tt_x46x6d62lws668qnr0000gn/T/ipykernel_58251/391251084.py:42: DeprecationWarning: Bitwise inversion '~' on bool is deprecated. This returns the bitwise inversion of the underlying int object and is usually not what you expect from negating a bool. Use the 'not' operator for boolean negation or ~int(x) if you really want the bitwise inversion of the underlying int.
  if ~dict['isOff']:


In [122]:
cols = settings.columns[2:]
settings_col = cs.by_name(*cols)
settings.filter(~pl.all_horizontal(settings_col.is_null())).head()

In [129]:
def process_device_settings(settings):
    """
    Process the device settings json file and return a dictionary with the important keys
    """
    dfs = []
    for dict in settings:
        if (len(dict.keys()) == 2) and ('RecordInfo' in list(dict.keys())) and ('TelemetryModuleInfo' in list(dict.keys())): continue # --> skip these entries, they have no useful information
        if (len(dict.keys()) == 2) and ('RecordInfo' in list(dict.keys())) and ('BatteryStatus' in list(dict.keys())): continue # --> skip these entries, they have no useful information
        if (len(dict.keys()) == 2) and ('RecordInfo' in list(dict.keys())) and ('SubjectInfo' in list(dict.keys())): continue # --> skip these entries, they have no useful information
        entry = {}
        for key, value in dict.items():
            if key == 'RecordInfo':
                entry['HostUnixTime'] = value['HostUnixTime']
                entry['SessionId'] = value['SessionId']
                
            elif key == 'GeneralData':
                entry['ActiveGroup'] = value['therapyStatusData']['activeGroup']
                entry['TherapyStatus'] = value['therapyStatusData']['therapyStatus']
                
            elif key == 'SensingConfig':
                for i, d in enumerate(value['timeDomainChannels']):
                    entry[f'channel_{i}_sensing'] = {k: v for k, v in d.items() if k in ["hpf", "lpf1", "lpf2", "minusInput", "plusInput", "sampleRate"]}
                    
            elif key == 'DetectionConfig':
                ld0 = value['Ld0']
                ld0 = {k: [v] if isinstance(v, list) else v for k, v in ld0.items()}
                entry['Ld0'] = ld0
                
                ld1 = value['Ld1']
                ld1 = {k: [v] if isinstance(v, list) else v for k, v in ld1.items()}
                entry['Ld1'] = ld1
                
            elif 'TherapyConfigGroup' in key:
                if key == f'TherapyConfigGroup{entry.get("ActiveGroup")}':
                    entry['RateInHz'] = value['rateInHz']
                    entry['rampTime'] = value['rampTime']
                    for program in value['programs']:
                        if program['isEnabled'] == 0 and program["valid"]:
                            entry['pulseWidthInMicroseconds'] = program['pulseWidthInMicroseconds']
                            entry['AmplitudeInMilliamps'] = program['amplitudeInMilliamps']
                            for i, dict in enumerate(program['electrodes']['electrodes']):
                                if ~dict['isOff']:
                                    entry['stim_electrode'] = i
                            entry['active_recharge'] = program['miscSettings']['activeRechargeRatio']
            
            elif key == 'AdaptiveConfig':
                entry['adaptiveMode'] = value['adaptiveMode']
                entry['adaptiveStatus'] = value['adaptiveStatus']
                entry['delta_fall'] = value['deltas'][0]['fall']
                entry['delta_fall'] = value['deltas'][0]['fall']
                for sub_key in value.keys():
                    if 'state' in sub_key:
                        sub_dict = {'prog0AmpInMilliamps': value[sub_key]['prog0AmpInMilliamps'], 'rateTargetInHz': value[sub_key]['rateTargetInHz'], 'isValid': value[sub_key]['isValid']}
                        entry[f"Adaptive_{sub_key}"] = sub_dict
            
        if entry:
            entry = {k: [v] for k, v in entry.items()}
            dfs.append(pl.from_dict(entry))
        
    return pl.concat(dfs, how='diagonal')
        

In [131]:
event_file = '/Users/claysmyth/Desktop/Starr/RCS_Hypno_Data/RCS02/RCS02R/8_17_21/Session1629181419311/DeviceNPC700404H/EventLog.json'
with open(event_file, 'r') as file:
    event_log = json.load(file)

In [135]:
event_df = parse_event_log(event_log)

In [138]:
event_df.head()

SessionId,HostUnixTime,EventType,EvenSubType
str,i64,str,str
"""1629181419311""",1629181440821,"""Mirror Event L…","""08_16_2021 11:…"
"""1629181419311""",1629181502917,"""Log Data Succe…","""08_16_2021 11:…"
"""1629181419311""",1629181530925,"""Log Data Succe…","""08_16_2021 11:…"
"""1629181419311""",1629181531284,"""Adaptive On""","""08_16_2021 11:…"


In [134]:
def parse_event_log(event_log):
    """
    Parse the event log json file and return polars dataframe with relevant log info. NOTE: IGNORES BATTERY LOG INFO
    """
    rows = {'SessionId': [], 'HostUnixTime': [], 'EventType': [], 'EvenSubType': []}
    for dict in event_log:
        event = dict['Event']
        if ('LeadLocation' in event['EventType']) or ('Battery' in event['EventType']) or ('Application Version' in event['EventType']):
            continue
        else:
            rows['SessionId'].append(dict['RecordInfo']['SessionId'])
            rows['HostUnixTime'].append(dict['RecordInfo']['HostUnixTime'])
            rows['EventType'].append(event['EventType'])
            rows['EvenSubType'].append(event['EventSubType'])
    
    return pl.DataFrame(rows)

In [139]:
adapt_file = '/Users/claysmyth/Desktop/Starr/RCS_Hypno_Data/RCS02/RCS02R/8_18_21/Session1629269941495/DeviceNPC700404H/AdaptiveLog.json'
with open(adapt_file, 'r') as file:
    adapt_log = json.load(file)

In [140]:
adapt_df = parse_adaptive_log(adapt_log)

In [141]:
adapt_df.head()

SessionId,HostUnixTime,PacketGenTime,PacketRxUnixTime,AdaptiveState,StimRateInHz,AmplitudeInMilliamps,Ld0DetectionStatues,Ld1DetectionStatues
str,i64,i64,i64,i64,f64,f64,i64,i64
"""1629269941495""",1629270050701,-62135568000000,1629270050701,0,130.2,2.7,0,0
"""1629269941495""",1629270051204,1629270051150,1629270051204,0,130.2,2.7,0,0
"""1629269941495""",1629270051699,1629270051647,1629270051699,0,130.2,2.7,0,0
"""1629269941495""",1629270052198,1629270052153,1629270052198,0,130.2,2.7,0,0
"""1629269941495""",1629270052693,1629270052652,1629270052693,0,130.2,2.7,0,0


In [18]:
def parse_adaptive_log(adaptive_log):
    """
    Parse the adaptive log json file and return polars dataframe with relevant log info
    """
    rows = {'SessionId': [], 'HostUnixTime': [], 'PacketGenTime': [], 'PacketRxUnixTime': [], 'AdaptiveState': [], 'StimRateInHz': [], 'AmplitudeInMilliamps': [], 'Ld0DetectionStatues': [], 'Ld1DetectionStatues': []}
    for dict in adaptive_log:
        adapt = dict['AdaptiveUpdate']
        rows['SessionId'].append(dict['RecordInfo']['SessionId'])
        rows['HostUnixTime'].append(dict['RecordInfo']['HostUnixTime'])
        rows['PacketGenTime'].append(adapt['PacketGenTime'])
        rows['PacketRxUnixTime'].append(adapt['PacketRxUnixTime'])
        rows['AdaptiveState'].append(adapt['CurrentAdaptiveState'])
        rows['StimRateInHz'].append(adapt['StimRateInHz'])
        rows['AmplitudeInMilliamps'].append(adapt['CurrentProgramAmplitudesInMilliamps'][0])
        rows['Ld0DetectionStatues'].append(adapt['Ld0DetectionStatus'])
        rows['Ld1DetectionStatues'].append(adapt['Ld1DetectionStatus'])
    
    return pl.DataFrame(rows)